# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv('JLW.env')


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jwales/nltk_data...


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 101


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'https://www.engadget.com/about/editors/mariella-moon',
 'title': "Facebook's Diem Association shifts its focus to the US",
 'description': "Diem has withdrawn its application for a Swiss payment license from Switzerland's FINMA financial regulator..",
 'url': 'https://www.engadget.com/facebook-diem-association-us-111008891.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-images/2019-10/2e15b870-e49b-11e9-97df-0bff4ac4372f',
 'publishedAt': '2021-05-13T11:20:08Z',
 'content': "The Facebook-backed Diem Association has revealed its plans to launch a stablecoin backed by the US dollar as it shifts its focus to the United States. Formerly known as Facebook's Libra Association,… [+1597 chars]"}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2021-05-13,The Facebook-backed Diem Association has revea...,0.4404,0.112,0.0,0.888
1,2021-05-12,"Digital currency group Diem Association, forma...",0.4215,0.080,0.0,0.920
2,2021-05-11,"Mark Zuckerberg posted a picture of his two, f...",0.8455,0.217,0.0,0.783
3,2021-05-13,Small toy figures stand on representations of ...,0.0000,0.000,0.0,1.000
4,2021-05-13,By Reuters Staff\r\n(The author is a Reuters B...,0.0000,0.000,0.0,1.000


In [1]:
libra_df[0]

NameError: name 'libra_df' is not defined

In [7]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.175179,0.071830,0.030180,0.897990
std,0.335714,0.070055,0.045047,0.091837
min,-0.599400,0.000000,0.000000,0.624000
25%,0.000000,0.000000,0.000000,0.838000
50%,0.051350,0.070000,0.000000,0.920500
75%,0.421500,0.094000,0.065250,1.000000
max,0.894100,0.257000,0.169000,1.000000
